In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import nglview as nv
#import warnings
#warnings.filterwarnings('ignore')

# Working with NGLView

In [3]:
molecular_system = msm.convert([nv.datafiles.GRO, nv.datafiles.XTC], to_form='molsysmt.MolSys')

In [4]:
view = nv.show_molsysmt(molecular_system)

In [5]:
view

NGLWidget(max_frame=50)

In [6]:
msm.view(molecular_system, standard=True)

NGLWidget(max_frame=50)

In [7]:
msm.info(view)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_structures
nglview.NGLWidget,5547,349,123,1,123,1,1,51


In [8]:
msm.get(view, element='group', selection=[81, 82, 83], name=True)

array(['VAL', 'ALA', 'ASP'], dtype=object)

In [9]:
msm.get(view, element='system', n_structures=True)

51

In [10]:
msm.get(view, element='atom', selection='atom_name=="CA"', coordinates=True)

Magnitude,[[[3.722000122070312 4.478000164031982 1.1200000047683716] [3.968000173568725 4.531000137329102 1.4120000600814817] [3.868000268936157 4.228000164031982 1.6150001287460325] ... [3.529000282287597 3.6380002498626705 8.02400016784668] [3.3240001201629634 3.9370002746582027 7.934000492095946] [3.1470000743865962 4.021000385284424 8.261000633239746]] [[3.788000106811523 4.4720001220703125 1.125] [4.02400016784668 4.539000034332275 1.4180001020431516] [3.911000251770019 4.2260003089904785 1.6060000658035276] ... [3.3730001449584956 3.783000230789184 8.10300064086914] [3.290000200271606 4.13700008392334 7.985000610351562] [3.4320001602172847 4.276000022888184 8.314000129699707]] [[3.8150002956390376 4.578999996185303 1.0800000429153442] [4.0420002937316895 4.584000110626221 1.392000079154968] [3.904000282287597 4.266000270843506 1.5590000152587888] ... [3.294000148773193 3.9360001087188716 8.0600004196167] [3.357000112533569 4.295000076293945 7.944000244140624] [3.337000131607055 4.50100040435791 8.266000747680664]] ... [[3.66700005531311 4.51800012588501 1.227000117301941] [3.9630002975463863 4.493000030517578 1.4770001173019407] [3.865000247955322 4.159000396728516 1.6440000534057615] ... [3.541000127792358 4.144000053405762 7.59600019454956] [3.4530000686645503 4.319000244140625 7.934000492095946] [3.3210000991821285 4.636000156402588 7.757000446319579]] [[3.8780002593994136 4.656000137329102 1.152999997138977] [4.076000213623047 4.534000396728516 1.455000042915344] [3.8960001468658443 4.2230000495910645 1.6030000448226926] ... [3.41700005531311 4.187000274658203 7.532000541687011] [3.225000143051147 4.458000183105469 7.717000484466552] [3.295000076293945 4.781000137329102 7.513000488281249]] [[3.6810002326965328 4.587000370025635 1.1299999952316284] [3.9560000896453853 4.549000263214111 1.3860000371932981] [3.8940002918243404 4.205000400543213 1.549000024795532] ... [3.3060002326965328 4.082000255584717 7.410000324249267] [3.0750000476837154 4.258000373840332 7.6680002212524405] [3.162000179290771 4.6060004234313965 7.538000583648681]]]
Units,nanometer


In [11]:
msm.get(view, element='system', box=True) # Warning: only 1 frame

Magnitude,[[[8.092 0.0 0.0] [0.0 8.092 0.0] [0.0 0.0 9.651900000000001]]]
Units,nanometer


In [12]:
msm.is_composed_of(view, proteins=1)

True

In [13]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"')

array([ 226, 1053, 1075, 2235, 3652, 3851, 3898, 4965, 5214, 5405])

In [14]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"', to_syntax='nglview')

'@226,1053,1075,2235,3652,3851,3898,4965,5214,5405'

In [15]:
msm.select(view, element='group', selection='group_name=="LYS"', to_syntax='nglview')

'16:0 66:0 67:0 141:0 231:0 245:0 248:0 311:0 325:0 339:0'

In [16]:
contact_map = msm.structure.get_contacts(view, selection='molecule_type=="protein" and atom_name=="CA"', threshold='12 angstroms')

In [19]:
labels = msm.get_label(view, element='atom', selection='molecule_type=="protein" and atom_name=="CA"', string='{group_id}')

In [20]:
CA_labels = msm.get_label(view, selection='molecule_type=="protein" and atom_name=="CA"', string='{group_id}')

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(contact_map[10], cmap='Greys', origin='lower')
plt.show()

In [ ]:
msm.convert(view, to_form='string:aminoacids3')

In [ ]:
molsys_A = msm.build.build_peptide('AceAlaNME')
molsys_B = msm.structure.translate(molsys_A, translation='[0.5, 0.0, 0.0] nm')

In [ ]:
view1 = msm.convert(molsys_A, to_form='nglview.NGLWidget')
view2 = msm.view(molsys_B)

In [ ]:
view = msm.merge([view1, view2])

In [ ]:
msm.info(view)

In [ ]:
msm.structure.get_geometric_center(view, selection='molecule_index==0')